## Download GEE: Elevation and slope
This script aims to download automatically Elevation and Slope data that match the images of the DRIVENDATA competition. Assuming that you have a google earth engine account, each image will be downloaded into a Google Drive folder of the same account.

In [41]:
import os
from osgeo import gdal, ogr, osr
import ee
print(ee.__version__)
from time import sleep

0.1.279


In [42]:
ee.Initialize()

Defining path where images are located

In [43]:
path_root = r'D:\DOCTORATE\Competitions\Drivendata\data\final\public\train_labels'
folder_drive = 'Doctorate'

Functions definition:

In [44]:
def get_polygon(path):
    #Getobject
    rast_data_source = gdal.Open(path)
    gt = rast_data_source.GetGeoTransform()
    #Raster info
    minx = gt[0]
    maxy = gt[3]
    maxx = minx + gt[1] * rast_data_source.RasterXSize
    miny = maxy + gt[5] * rast_data_source.RasterYSize
    wkt = f'POLYGON (({minx} {maxy},{maxx} {maxy},{maxx} {miny},{minx} {miny},{minx} {maxy}))'
    poly = ogr.CreateGeometryFromWkt(wkt)
    #spatial reference
    native_projection = osr.SpatialReference()
    native_projection.ImportFromWkt(rast_data_source.GetProjection())
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)
    transform = osr.CoordinateTransformation(native_projection, target)
    poly.Transform(transform)
    #get reprojected coordinates
    pt = []
    for fea in range(poly.GetGeometryRef(0).GetPointCount()):
        lat,long,h = poly.GetGeometryRef(0).GetPoint(fea)
        pt.append([long,lat])
    #Create an ee.Geometry
    polygon = ee.Geometry.Polygon(pt)
    #reference system number of source layer
    Epsg = native_projection.GetAttrValue("AUTHORITY", 0)+":" + native_projection.GetAttrValue("AUTHORITY", 1) 
    rast_data_source = None
    return polygon, Epsg
    
def get_gee_elevation(name_file,polygon, Epsg):
    #get DEM layer
    elevation = ee.Image("USGS/SRTMGL1_003").select('elevation').clip(polygon)
    elevation1 = elevation.resample('bilinear').reproject(crs=Epsg, scale=10)
    
    #save data
    task_config = {
        'folder': folder_drive,
        'fileFormat': 'GeoTIFF',
        'region' : polygon,
        'dimensions': 512
    }
    name_file_inf = name_file + "_dem"
    task = ee.batch.Export.image.toDrive(elevation1, name_file_inf, **task_config)
    task.start()
    k = 1
    #sleep loop until data is downloaded
    while task.status()['state'] == 'COMPLETED':
        sleep(5)
        k += 1
        print(f'Iteration Elevation: {k}')
    elevation = None
    task = None

#Here, images are downloade in 16 bits
def get_gee_slope(name_file,polygon, Epsg):
    #get Slope layer
    elevation = ee.Image("USGS/SRTMGL1_003").select('elevation').clip(polygon)
    slope = ee.Terrain.slope(elevation)
    slope1 = slope.resample('bilinear').reproject(crs=Epsg, scale=10).multiply(10).toShort() 
    #save data
    task_config = {
        'folder': folder_drive,
        'fileFormat': 'GeoTIFF',
        'region' : polygon,
        'dimensions': 512
    }
    name_file_inf = name_file + "_slp"
    task = ee.batch.Export.image.toDrive(slope1, name_file_inf, **task_config)
    task.start()
    k = 1
    #sleep loop until data is downloaded
    while task.status()['state'] == 'COMPLETED':
        sleep(5)
        k += 1
        print(f'Iteration slope: {k}')
    slope1 = None
    task = None

Iteration through image files

In [ ]:
for i in os.listdir(path_root):
    if i.endswith('.tif'):
        path = os.path.join(path_root,i)
        polygon, Epsg = get_polygon(path)
        #get DEM
        name_file = i.split('.tif')[0]
        get_gee_elevation(name_file, polygon, Epsg)
        #get Slope
        get_gee_slope(name_file, polygon, Epsg)
        print(f'Done with image {name_file}')

Done with image awc00
Done with image awc01
Done with image awc02
Done with image awc03
